## Explanation of Each Layer
Layer	Input	Output / Action
Detection (GRU)	AIS sequence (LAT, LON, SOG, ...)	prob → probability of spoofing (0..1)
ResponseModule	vessel_id, lat, lon, prob	Log all predictions in CSV, email only confirmed
Deployment / Inference	Synthetic or real AIS sequences	Feed sequence → GRU → ResponseModule
Email / Notification	Confirmed alerts from ResponseModule	Email alert sent via Apps Script

In [5]:


!pip install pandas numpy tensorflow requests


import os
import csv
import datetime
import numpy as np
from collections import defaultdict
from tensorflow.keras.models import load_model


model_gru = load_model("/content/drive/MyDrive/gru_spoofing_detector.h5")
print("✅ GRU model loaded.")


class ResponseModule:
    def __init__(self, log_file="/content/spoofing_alerts.csv", apps_script_url=None):
        self.log_file = log_file
        self.apps_script_url = apps_script_url
        if not os.path.exists(self.log_file):
            with open(self.log_file, "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(["Timestamp","Vessel_ID","Latitude","Longitude","Prob","Action","Note"])
        self.counters = defaultdict(int)
        self.threshold = 0.5
        self.confirmation_required = 2
    def log_alert(self, vessel_id, lat, lon, prob, action="LOGGED", note=""):
        with open(self.log_file, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([datetime.datetime.utcnow().isoformat(),
                             vessel_id, lat, lon, float(prob), action, note])
        print(f"[LOG] {vessel_id} | {lat:.6f},{lon:.6f} | prob={prob:.3f} | {action} | {note}")

    def send_email(self, vessel_id, lat, lon, prob):
        if not self.apps_script_url:
            print("[EMAIL] Apps Script URL not set. Skipping email.")
            return
        import requests
        try:
            requests.get(self.apps_script_url, params={
                "vesselId": vessel_id,
                "lat": lat,
                "lon": lon,
                "prob": prob
            })
            print("[EMAIL] Alert sent via Apps Script.")
        except Exception as e:
            print("[EMAIL] Error sending alert:", e)

    def handle_prediction(self, vessel_id, lat, lon, prob):

        if prob >= self.threshold:
            self.counters[vessel_id] += 1
        else:
            self.counters[vessel_id] = 0

        if self.counters[vessel_id] >= self.confirmation_required:
            note = "Confirmed after debounce"
            self.log_alert(vessel_id, lat, lon, prob, action="CONFIRMED", note=note)
            self.send_email(vessel_id, lat, lon, prob)
            self.counters[vessel_id] = 0
        else:

            self.log_alert(vessel_id, lat, lon, prob, action="PENDING", note=f"count={self.counters[vessel_id]}")


apps_script_url = "https://script.google.com/macros/s/AKfycbzYc28lg-WsIO0ljA5iE87HZSUGIaUuIrZPWTZj9cSSIVMz3ie9q9Qg6RM9IpoMa1doLg/exec"  # replace with your Google Apps Script URL
response_module = ResponseModule(apps_script_url=apps_script_url)


num_sequences = 5
seq_length = 10
num_features = 21

for i in range(num_sequences):
    vessel_id = f"Vessel_{100+i}"
    lat = np.random.uniform(-90, 90)
    lon = np.random.uniform(-180, 180)


    for j in range(2):
        X_seq = np.random.rand(seq_length, num_features).astype(np.float32)


        if i % 2 == 0:
            prob = 0.9
        else:

            prob = model_gru.predict(X_seq.reshape(1, seq_length, num_features))[0][0]

        response_module.handle_prediction(vessel_id, lat, lon, prob)


/tmp/ipython-input-1058427225.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  writer.writerow([datetime.datetime.utcnow().isoformat(),


✅ GRU model loaded.
[LOG] Vessel_100 | -89.557415,-34.669048 | prob=0.900 | PENDING | count=1
[LOG] Vessel_100 | -89.557415,-34.669048 | prob=0.900 | CONFIRMED | Confirmed after debounce
[EMAIL] Alert sent via Apps Script.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
[LOG] Vessel_101 | 80.440654,-99.884246 | prob=1.000 | PENDING | count=1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[LOG] Vessel_101 | 80.440654,-99.884246 | prob=1.000 | CONFIRMED | Confirmed after debounce
[EMAIL] Alert sent via Apps Script.
[LOG] Vessel_102 | 29.037097,-28.538415 | prob=0.900 | PENDING | count=1
[LOG] Vessel_102 | 29.037097,-28.538415 | prob=0.900 | CONFIRMED | Confirmed after debounce
[EMAIL] Alert sent via Apps Script.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[LOG] Vessel_103 | -55.837240,175.502303 | prob=1.000 | PENDING | count=1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
[LOG] Vessel_103 | -55.837240,175.502303 | prob=1.000 | CONFIRMED | Confirmed after debounce
[EMAIL] Alert sent via Apps Script.
[LOG] Vessel_104 | 